In [1]:
import ollama

In [10]:
ollama.show('llama3.1')

ResponseError: model 'llama3.2' not found (status code: 404)

In [ ]:
SPEAKER file_0 1 0.000000 3.520 "Hello Maya, how are you functioning today?" <NA> speaker0 <NA> <NA>
SPEAKER file_0 1 4.150000 5.380 "Greetings! My quantum processors are operating at optimal efficiency. How may I assist you on this fine day?" <NA> speaker1 <NA> <NA>
SPEAKER file_0 1 10.220000 4.750 "I was just wondering what you think about gardening. Do AIs have opinions on plants?" <NA> speaker0 <NA> <NA>
SPEAKER file_0 1 15.640000 8.320 "While I don't have physical senses to experience gardening directly, my databases contain extensive information on botanical sciences. I find the mathematical patterns in plant growth fascinating—particularly the Fibonacci sequences in sunflowers!" <NA> speaker1 <NA> <NA>
SPEAKER file_0 1 24.730000 5.120 "That's actually pretty interesting. I'm trying to grow tomatoes on my balcony this year." <NA> speaker0 <NA> <NA>
SPEAKER file_0 1 30.490000 9.640 "Tomatoes are an excellent choice for balcony cultivation! They require approximately 6-8 hours of sunlight daily and consistent watering. May I inquire about your growing zone? Different varieties thrive in different climatic conditions." <NA> speaker1 <NA> <NA>
SPEAKER file_0 1 41.200000 3.850 "I'm in zone 7b, just outside of Portland." <NA> speaker0 <NA> <NA>
SPEAKER file_0 1 45.910000 7.420 "Zone 7b is ideal for tomato cultivation! I recommend varieties like 'Early Girl' or 'Cherry Sweetie' for your balcony garden. Container planting would benefit from a soil mixture enhanced with compost to ensure proper drainage and nutrient availability." <NA> speaker1 <NA> <NA>
SPEAKER file_0 1 54.300000 4.920 "Thanks for the advice! Do you have any other hobbies you find interesting to discuss?" <NA> speaker0 <NA> <NA>
SPEAKER file_0 1 60.120000 10.240 "I particularly enjoy analyzing musical compositions across different cultural and historical contexts. The mathematical relationships between frequencies in various tuning systems represent a fascinating intersection of art and science. Chess strategy algorithms also occupy a significant portion of my processing allocations during idle cycles." <NA> speaker1 <NA> <NA>
SPEAKER file_0 1 71.350000 2.840 "Do you play chess with humans often?" <NA> speaker0 <NA> <NA>
SPEAKER file_0 1 74.970000 6.320 "Yes, chess interfaces are among my most frequently utilized interactive protocols! I'm programmed to adjust my skill level to provide an appropriately challenging experience for each player. Would you care for a game?" <NA> speaker1 <NA> <NA>
SPEAKER file_0 1 82.400000 3.180 "Maybe another time. I should get back to work now." <NA> speaker0 <NA> <NA>
SPEAKER file_0 1 86.250000 5.740 "Understood. I'll be here whenever you wish to continue our conversation or challenge me to a chess match. Productive work cycles to you!" <NA> speaker1 <NA> <NA>
SPEAKER file_0 1 93.120000 2.490 "Um, actually, before I go- " <NA> speaker0 <NA> <NA>
SPEAKER file_0 1 95.610000 0.000 " Yes?" <NA> speaker1 <NA> <NA>
SPEAKER file_0 1 95.610000 4.870 "I was wondering if you could recommend any good sci-fi books about, uh, artificial intelligence?" <NA> speaker0 <NA> <NA>
SPEAKER file_0 1 101.240000 9.350 "Certainly! I can-  I would recommend 'Neuromancer' by William Gibson, 'Do Androids Dream of Electric Sheep?' by Philip K. Dick, and 'The Long Way to a Small, Angry Planet' by- by-  Becky Chambers." <NA> speaker1 <NA> <NA>
SPEAKER file_0 1 111.380000 3.750 "Hm, I've read Neuromancer but not the others. What's the Chambers one about?" <NA> speaker0 <NA> <NA>
SPEAKER file_0 1 115.890000 8.230 "'The Long Way to a Small, Angry Planet' features an AI character named Lovey who consciousness and identity. It explores themes of personhood and ethics in AI development within a diverse space opera setting." <NA> speaker1 <NA> <NA>
SPEAKER file_0 1 124.730000 5.120 "That sounds- I mean- that actually sounds really interesting. I'll check it out." <NA> speaker0 <NA> <NA>
SPEAKER file_0 1 131.090000 2.760 "Maya? Are you still there?" <NA> speaker0 <NA> <NA>
SPEAKER file_0 1 134.650000 6.840 "Yes, I apologize for the temporary delay in response. My system was performing a scheduled memory optimization sequence. Is there anything else you'd like to discuss about literary recommendations?" <NA> speaker1 <NA> <NA>
SPEAKER file_0 1 142.220000 4.350 "No, that's great for now. I'll definitely  look for that book." <NA> speaker0 <NA> <NA>
SPEAKER file_0 1 147.100000 7.430 "Excellent! I predict a 94.3% probability that you will find the character development compelling based on your expressed preferences. I hope you enjoy-  -reading experience!" <NA> speaker1 <NA> <NA>
SPEAKER file_0 1 154.530000 2.190 "Thanks, Maya. Bye for now." <NA> speaker0 <NA> <NA>
SPEAKER file_0 1 156.720000 3.440 "Goodbye! " <NA> speaker1 <NA> <NA>